# Cloud Pendulum Tutorial

Here is a simple example showing how to use the Cloud Pendulum system. This is a system that allows users to remotely run experiments on a single motor Double Integrator hardware. The clients requests a "cell" to run their experiment on, and the server gives access to these cells whenever one is available. There are many types of experiments you can run on this system (e.g. simple pendulum, double pendulum, acrobot, pendubot etc.), each requiring a specific type of hardware in the cell. In this tutorial we will be using the `DoubleIntegrator` experiment type, which runs on cells that contain a single motor, like the one in the following picture:

<div>
<img src="media/double_integrator.jpeg" width="250"/>
</div>

In this tutorial, a couple of simple to advanced topics will be shown. This includes
- how to connect to a system and start an experiment
- how to collect data (and visualize it in the notebook)
- how to control a system using the built-in impendance controller. You can set
    - position
    - velocity
    - torque
    - a mix between them
- some implementation details regarding accuracy and real-time control.

# Pre-requisite

The first step is to create an account on the cloud pendulum JupyterHub: https://cloudpendulum.m2.chalmers.se:1443/hub/signup. Once you sign up, contact one of the administrators to authorize your account. Once we authorize, you will be able to login into the system and write your own code for example, related to modeling, simulation, learning and control. During the authorization, you will also be provided with a user token with which you can do experiments on the real hardware. 

# Setup

To communicate with the server, we use the `cloudpendulum` client package. The documentation for this package can be found here: https://cloudpendulum.m2.chalmers.se/docs/.

In [ ]:
from cloudpendulumclient.client import Client

The interface is built around the `Client` class, which handles the connection to the server. To start an experiment, the `start_experiment` method is used to ask the server to reserve a 'cell' for some period of time. We supply a `user_token` to this method which is used by the server to authenticate the user.

In [ ]:
user_token = "YOUR USER TOKEN HERE"

You can find your user token in the 'user_token' by visiting File->Hub Control Panel->CP Token & Compute Status. To check your token status, you can use either the `get_user_info()` method or visit `File->Hub Control Panel->CP Token & Compute Status`. 

In [ ]:
client = Client()
client.get_user_info(user_token) # Monitor the status of your user token

### Starting an experiment

Let's try to start an experiment:

In [ ]:
import time

client = Client()
session_token, livestream_url = client.start_experiment(
    user_token = user_token,
    experiment_type = "DoubleIntegrator",
    experiment_time = 5.0,
    preparation_time = 5.0,
    record = True
)
print("Received response from server!")
print("Session token: ", session_token)
print("Livestream url: ", livestream_url)
print("Sleeping...")

time.sleep(3.0)
print("Stopping experiment")
video_url = client.stop_experiment(session_token)
time.sleep(1.0)

# Show video
from misc import download_video
video_path = download_video(video_url)
from IPython.display import Video
Video(video_path)

When running the `start_experiment`, you may have received a response immediately, or it may have taken a while. The reason for this is that the server will wait to send the reply until it has successfully found an appropriate cell available to reserve for the client. If many clients are using the system simultaneously, some may have to wait for their turn.

From the `start_experiment` method, we get a session token and a livestream url. The session token is used by the server to authenticate all subsequent request relating to this experiment session, and the livestream url is used to view a live video feed of the experiment. The `stop_experiment` method is used to stop the experiment, as you would expect. If we had not used the `stop_experiment` method, the experiment would have stopped by itself after 5 seconds.

Since we specified `record = True` when starting the experiment, we get a video url from `stop_experiment` so that we can watch the experiment.

### Reading the pendulum states

We can expand this example a bit and read the position, velocity and torque values: We do this by using the `get_position`, `get_velocity` and `get_torque` methods.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

Tf = 5.0
session_token, livestream_url = client.start_experiment(
    user_token = user_token,
    experiment_type = "DoubleIntegrator",
    experiment_time = Tf,
    preparation_time = 5.0,
    initial_state = [0.0, 0.0],
    record = False
)
print("Received response from server!")
print("Session token: ", session_token)
print("Livestream url: ", livestream_url)

dt = 0.01
meas_dt = 0.0
meas_time = 0.0

print("Desired dt: ", dt)

n = int(Tf / dt)

# Create data matrices
meas_time_vec = np.zeros(n)
meas_pos = np.zeros(n)
meas_vel = np.zeros(n)
meas_tau = np.zeros(n)
meas_dt_vec = np.zeros(n)

i = 0

print("Running the control loop")
while meas_time < Tf and i < n:
    start_loop = time.time()
    meas_time += meas_dt

    # Measure data
    measured_position = client.get_position(session_token) # Measure position
    measured_velocity = client.get_velocity(session_token) # Measure velocity
    measured_torque = client.get_torque(session_token) # Measure torque

    # add data to matrices
    meas_time_vec[i] = meas_time
    meas_pos[i] = measured_position
    meas_vel[i] = measured_velocity    
    meas_tau[i] = measured_torque
        
    while time.time() - start_loop < dt:
        pass
        
    meas_dt = time.time() - start_loop
    meas_dt_vec[i] = meas_dt

    i = i + 1
    
print("Finished")
print("Average measured dt: ", np.mean(meas_dt_vec))

# Show plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

# Measured Position
ax1.plot(meas_time_vec, meas_pos)
ax1.set_xlabel("Time (s)") 
ax1.set_ylabel("Position (rad)")
ax1.set_title("Measured Position (rad) vs Time (s)")

# Measured Velocity
ax2.plot(meas_time_vec, meas_vel)
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (rad/s)")
ax2.set_title("Measured Velocity (rad/s) vs Time (s)")

# Measured Torque
ax3.plot(meas_time_vec, meas_tau)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Torque (Nm)")
ax3.set_title("Measured Torque (Nm) vs Time (s)")

plt.tight_layout()
plt.show()

From this example, we can read a couple of things:

First of all, we see a baseline for the noise that we have in the system. For the position, we can as well see the discretization of the sensor at 0.0004 rad. Take a look and examine it yourself!

Also, we notice what looks like a problem with our plots at first: Additionally to the data, the plots show a line from the last data point to (0,0)! 

It turns however out that our plots and code are fine. The issue is instead related to the difficulties with doing real-time control on computers that are not made for real-time control: 
- In each control loop iteration, we first perform our calculations (reading sensors, computing control signals, etc.), which typically completes faster than the sampling time. We then have to wait for the remainder of the sampling period to actually pass before starting the next iteration. 
- We have preallocated our data matrices with $n = \frac{t_{\text{experiment}}}{t_{\text{sampling}}}$ entries. So if the controller operates for $t_{\text{experiment}}$ seconds, $n$ steps should be executed. The matrices should thus be fully filled up. 
- However, sometimes individual control steps take just a little bit longer than they should. These timing errors accumulate, meaning the while loop terminates based on the elapsed time before all $n$ steps are executed and the preallocated data fields are filled. 

We can verify this by printing the last 20 elements of the position and time data:

In [ ]:
print("Position vector")
print(meas_pos[-20:])
print("Time vector")
print(meas_pos[-20:])

For a nicer plot, we can clean up our data:

In [ ]:
# Data cleanup for plotting because sometimes a few control ticks maybe missed 
lastnonzero_index = np.max(np.nonzero(meas_time_vec)[-1]) + 1
N = int(Tf/dt)
print("Number of missed control ticks: ", N - lastnonzero_index)
meas_time_vec = meas_time_vec[:lastnonzero_index]
meas_pos = meas_pos[:lastnonzero_index]
meas_vel = meas_vel[:lastnonzero_index]
meas_tau = meas_tau[:lastnonzero_index]

# Show plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

# Measured Position
ax1.plot(meas_time_vec, meas_pos)
ax1.set_xlabel("Time (s)") 
ax1.set_ylabel("Position (rad)")
ax1.set_title("Measured Position (rad) vs Time (s)")

# Measured Velocity
ax2.plot(meas_time_vec, meas_vel)
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (rad/s)")
ax2.set_title("Measured Velocity (rad/s) vs Time (s)")

# Measured Torque
ax3.plot(meas_time_vec, meas_tau)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Torque (Nm)")
ax3.set_title("Measured Torque (Nm) vs Time (s)")

plt.tight_layout()
plt.show()

In general, it is hard to get exact timing on standard operating systems. In our case, we do not have a real-time OS at hand, but we can at least tune our computer. One way of achieving better real-time performance is e.g. the following workaround:

In [ ]:
def wait_for_control_loop_end(start_loop, dt):
        """Delay ending a while loop so that it loops at a desired sampling time dt."""
        time_to_pass = start_loop + dt - time.time()
        if time_to_pass <= 0.0:
            return
        
        time.sleep(time_to_pass * 0.7) # sleep
        while time.time() - start_loop < dt: # busy waiting
            pass

In future experiments, we use this method for better accuracy in timing. At this point, we do not discuss the how/why in detail, but feel free to check the detailed analysis later in this tutorial.

# Controlling the motor

Now that we have reserved a cell from the server and read the data, let's do what we are here for: Let's control the motor! The controller that the API provides us is an impendance controller. The following excerpt from the documentation of the hardware describes its functionality very well:

From [MAB documentation](https://mabrobotics.github.io/MD80-x-CANdle-Documentation/MD/motion.html#impedance-pd):

Impedance Control mode is a popular choice for mobile or legged robots, as well as for any compliant mechanism. The main idea behind it is to mimic the behavior of a torsional spring with variable stiffness and damping. The parameters of the controller are:
- Position Target
- Velocity Target
- kP (position gain)
- kD (velocity gain)
- Torque Feed Forward (Torque FF)

The torque output is proportional to the position error and velocity error and additionally supplemented with a torque command from the user. Here are some of the most common applications for this control mode:

- **Spring-damper mechanism** - when velocity target is set to 0, impedance controllers kP gain acts as the virtual spring stiffness and kD as its damping coefficient. Example use case: a variable suspension for a wheeled robot, where suspension stiffness can be regulated by kP, damping by kD, and height (clearance) by changing the target position;
- **High-frequency torque controller**, where its targets and gains can act as stabilizing agents to the torque command. Example use case: In legged robots, force control can be achieved by advanced control algorithms, which usually operate at rates below 100 Hz. It is usually enough to stabilize the robot but too slow to avoid vibrations. Knowing desired robot’s joint positions, velocities, and torques, drives can be set to produce the proper torque and hold the position/velocity with small gains. This would compensate for any high-frequency oscillations (vibrations) that may occur, as the impedance controller works at 40kHz (much faster than <100 Hz).
- **Raw torque controller** - when kP and kD are set to zero, the torque_ff command is equal to the output controller torque.
- **Idle** - when kP and kD are set to zero, and the torque_ff command is equal to zero, the motor shaft will be free to rotate. When the drive is disabled it connects all the windings together for safety. This mode can be useful for enabling free rotation of the shaft, but the rotational energy should not be too high as the voltages induced in the motor windings could break the driver.

The impedance controller is relatively simple and works according to the schematic below:

<div>
<img src="media/impedance_controller.png" width="800"/>
</div>

The controller can therefore be written as follows, and allows for a mixture of different applications:
$$u = K_p (\theta - \theta^*) + K_d (\dot{\theta} - \dot{\theta}^*) + \theta \tag{1},$$
where $\theta$ is the angle and $\dot{\theta}$ is the angular velocity.

The motors are by default started in impedance control mode with proportional gain Kp and derivative gain Kd set to zero i.e. pure torque control. In this example we use `set_position` to set the position of the motor directly. We set only $K_p$ and $K_d$, and a setpoint for the position. In this case, the impendance controller becomes a simple PD controller for the position.

Later, we will use `set_velocity` or `set_torque` to set the velocity and torque of the motor respectively.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

Tf = 2 * math.pi
client = Client()
session_token, livestream_url = client.start_experiment(
    user_token = user_token,
    experiment_type = "DoubleIntegrator",
    experiment_time = Tf,
    preparation_time = 5.0,
    initial_state = [0.0, 0.0],
    record = True
)
print("Received response from server!")
print("Session token: ", session_token)
print("Livestream url: ", livestream_url)

# Set Kp, Kd for position control
kp = 0.04
kd = 0.001
client.set_impedance_controller_params([kp], [kd], session_token)

dt = 0.01
meas_dt = 0.0
meas_time = 0.0

print("Desired dt: ", dt)

n = int(Tf / dt)

meas_time_vec = np.zeros(n)
meas_pos = np.zeros(n)
meas_vel = np.zeros(n)
meas_tau = np.zeros(n)
meas_dt_vec = np.zeros(n)

i = 0

print("Running the control loop")
while meas_time < Tf and i < n:
    start_loop = time.time()
    meas_time += meas_dt

    # Measure data
    measured_position = client.get_position(session_token) # Measure position
    measured_velocity = client.get_velocity(session_token) # Measure velocity
    measured_torque = client.get_torque(session_token) # Measure torque

    # Send position command
    client.set_position(math.sin(meas_time), session_token)

    # Collect data for plotting
    meas_time_vec[i] = meas_time
    meas_pos[i] = measured_position
    meas_vel[i] = measured_velocity    
    meas_tau[i] = measured_torque
        
    wait_for_control_loop_end(start_loop, dt)
    
    meas_dt = time.time() - start_loop
    meas_dt_vec[i] = meas_dt

    i = i + 1
    
print("Finished")

video_url = client.stop_experiment(session_token)
time.sleep(1.0)

print("Average measured dt: ", np.mean(meas_dt_vec))

# Show plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

# Measured Position
ax1.plot(meas_time_vec, meas_pos)
ax1.set_xlabel("Time (s)") 
ax1.set_ylabel("Position (rad)")
ax1.set_title("Measured Position (rad) vs Time (s)")

# Measured Velocity
ax2.plot(meas_time_vec, meas_vel)
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (rad/s)")
ax2.set_title("Measured Velocity (rad/s) vs Time (s)")

# Measured Torque
ax3.plot(meas_time_vec, meas_tau)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Torque (Nm)")
ax3.set_title("Measured Torque (Nm) vs Time (s)")

plt.tight_layout()
plt.show()

# Show video
from misc import download_video
video_path = download_video(video_url)
from IPython.display import Video
Video(video_path)

**Remark:** The position tracking is not perfect. By selecting better Kp and Kd values this can be improved.

## Velocity control

Next, we control the velocity $\dot{\theta}$ instead of the position $\theta$, and set a desired velocity $\dot{\theta}^*$. To only control the velocity, we disable the part controlling the position by setting $K_p=0$, and choose a desired $K_d$, essentially resulting in a P-controller for the velocity.

In this example, we also set the initial state of the experiment to $\left[ \frac{\pi}{2}, 0.1 \right]$ using the `initial_state` option when calling `client.start_experiment(...)`.

In [ ]:
Tf = 5.0
client = Client()
session_token, livestream_url = client.start_experiment(
    user_token = user_token,
    experiment_type = "DoubleIntegrator",
    experiment_time = Tf,
    preparation_time = 5.0,
    initial_state = [np.pi*0.5, 0.1],
    record = True
)
print("Received response from server!")
print("Session token: ", session_token)
print("Livestream url: ", livestream_url)

# Set Kp, Kd for velocity control
kp = 0.0
kd = 0.04
client.set_impedance_controller_params([kp], [kd], session_token)

dt = 0.01
meas_dt = 0.0
meas_time = 0.0

print("Desired dt: ", dt)

n = int(Tf / dt)

meas_time_vec = np.zeros(n)
meas_pos = np.zeros(n)
meas_vel = np.zeros(n)
meas_tau = np.zeros(n)
meas_dt_vec = np.zeros(n)

i = 0

print("Running the control loop")
while meas_time < Tf and i < n:
    start_loop = time.time()
    meas_time += meas_dt

    # Measure data
    measured_position = client.get_position(session_token) # Measure position
    measured_velocity = client.get_velocity(session_token) # Measure velocity
    measured_torque = client.get_torque(session_token) # Measure torque

    # Send position command
    client.set_velocity(10.0, session_token)

    # Collect data for plotting
    meas_time_vec[i] = meas_time
    meas_pos[i] = measured_position
    meas_vel[i] = measured_velocity    
    meas_tau[i] = measured_torque

    wait_for_control_loop_end(start_loop, dt)

    meas_dt = time.time() - start_loop
    meas_dt_vec[i] = meas_dt
    
    i = i + 1

print("Finished")

video_url = client.stop_experiment(session_token)
time.sleep(1.0)

print("Average measured dt: ", np.mean(meas_dt_vec))

# Show plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

# Measured Position
ax1.plot(meas_time_vec, meas_pos)
ax1.set_xlabel("Time (s)") 
ax1.set_ylabel("Position (rad)")
ax1.set_title("Measured Position (rad) vs Time (s)")

# Measured Velocity
ax2.plot(meas_time_vec, meas_vel)
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (rad/s)")
ax2.set_title("Measured Velocity (rad/s) vs Time (s)")

# Measured Torque
ax3.plot(meas_time_vec, meas_tau)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Torque (Nm)")
ax3.set_title("Measured Torque (Nm) vs Time (s)")

plt.tight_layout()
plt.show()

# Show video
from misc import download_video
video_path = download_video(video_url)
from IPython.display import Video
Video(video_path)

## Torque control

In the last example, direct control of the torque is demonstrated. As you can see from eq. (1), the torque command is a direct feedforward. Thus, we set both $K_p = K_d = 0$, and use `set_torque` to set the torque.

In [ ]:
Tf = 2*math.pi
client = Client()
session_token, livestream_url = client.start_experiment(
    user_token = user_token,
    experiment_type = "DoubleIntegrator",
    experiment_time = Tf,
    preparation_time = 5.0,
    record = True
)
print("Received response from server!")
print("Session token: ", session_token)
print("Livestream url: ", livestream_url)

# Set Kp, Kd for torque control
kp = 0.0
kd = 0.0
client.set_impedance_controller_params([kp], [kd], session_token)

dt = 0.01
meas_dt = 0.0
meas_time = 0.0

print("Desired dt: ", dt)

n = int(Tf / dt)

meas_time_vec = np.zeros(n)
meas_pos = np.zeros(n)
meas_vel = np.zeros(n)
meas_tau = np.zeros(n)
meas_dt_vec = np.zeros(n)
des_tau = np.zeros(n)

i = 0

print("Running the control loop")
while meas_time < Tf and i < n:
    start_loop = time.time()
    meas_time += meas_dt

    # Measure data
    measured_position = client.get_position(session_token) # Measure position
    measured_velocity = client.get_velocity(session_token) # Measure velocity
    measured_torque = client.get_torque(session_token) # Measure torque

    # Send position command
    des_tau[i] = 0.004*np.sin(meas_time)
    client.set_torque(des_tau[i], session_token)

    # Collect data for plotting
    meas_time_vec[i] = meas_time
    meas_pos[i] = measured_position
    meas_vel[i] = measured_velocity    
    meas_tau[i] = measured_torque
        
    wait_for_control_loop_end(start_loop, dt)
    
    meas_dt = time.time() - start_loop
    meas_dt_vec[i] = meas_dt

    i = i + 1
    
print("Finished")

video_url = client.stop_experiment(session_token)
time.sleep(1.0)

print("Average measured dt: ", np.mean(meas_dt_vec))

# Show plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

# Measured Position
ax1.plot(meas_time_vec, meas_pos)
ax1.set_xlabel("Time (s)") 
ax1.set_ylabel("Position (rad)")
ax1.set_title("Measured Position (rad) vs Time (s)")

# Measured Velocity
ax2.plot(meas_time_vec, meas_vel)
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (rad/s)")
ax2.set_title("Measured Velocity (rad/s) vs Time (s)")

# Measured Torque
ax3.plot(meas_time_vec, meas_tau)
ax3.plot(meas_time_vec, des_tau)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("Torque (Nm)")
ax3.legend(["measured torque", "desired torque"])
ax3.set_title("Measured Torque (Nm) vs Time (s)")

plt.tight_layout()
plt.show()

# Show video
from misc import download_video
video_path = download_video(video_url)
from IPython.display import Video
Video(video_path)

# Realtime control in Linux/Python

As discussed earlier, usual operating systems such as Linux or Windows have problems with hard real-time deadlines. This is due to the complexity of the system: To achieve a good performance, features like virtual memory, preemptive multitasking, and complex schedulers have been introduced. They maximize the utilization of the compute resources, but introduce unpredictable latencies.

As we have seen at the start of the tutorial, this can cause our fixed sampling time to vary and the actual sampling time to be a bit longer than desired. The CloudPendulum only provides the hardware and a server to run your code on - what code you run is up to you, so the question how you deal with the system constraints is also up to you.

In order to give some best practices and to quantify the performance, we made some experiments with the following pseudo code:
```
while t < t_overall:
    start_loop = time.time()
    
    # do control stuff

    time_that_passed = time.time() - start_loop
    time.sleep( dt - time_that_passed )
```
The resulting benchmark of the sampling times, with $dt=0.01s$ looks like this:

<div>
<img src="media/benchmark_sleep.png" width="700"/>
</div>
Additionally the controller missed the last 3 ticks.

As you can see, the difference is usually not too large. As pragmatic approch can therefore be to accept the delays and use a clean-up of the data before plotting. The corresponding code for clean-up is given at the end of this tutorial.

A different approach would be to following:
```
while t < t_overall:
    start_loop = time.time()
    
    # do control stuff

    while time.time() - start_loop < dt:
        pass
```
This is called _busy waiting_, because the process runs all times and checks at each clock cycle if the time has passed. Let us also benchmark this:

<div>
<img src="media/benchmark_sleep_busy.png" width="700"/>
</div>
Additionally, the controller missed the last 17 ticks.

This is interesting: For the vast majority of steps, the time step was followed very well. But for some steps, the sampling time was exceeded by a lot!

To understand why this happens, we need to take a look at the internal working principles of our operating system: A CPU core can only execute one process at a time (leaving multi-core CPUs out of consideration for now). The scheduler decides which process gets the priority and is executed on the CPU core at each time. This can be for example this Python program, some background process, or any other program running on the computer.

With busy waiting, the Python process checks at each clock cycle if the sampling time has passed. If only the Python process is executed, this leads to be best preformance that we can see in the majority of steps. However, the Python process keeps the CPU busy at all time and thus blocks other processes. At some point, the scheduler will therefore prioritize another process higher and "pause" the Python process. This is what happens when we see the large delays.

With the first approach instead, the `sleep()` function, something different happens: In this case, the Python process actively gives back the control of the CPU and notifies the scheduler when it would like to be continued to run. This prevents the large delays, because the scheduler "knows" when to run which process. However, it only guarantees to sleep _at least_ for the desired amount of time, and in practice is shows to usually be a bit longer.

The `wait_for_control_loop_end` function, that was used above and is implemented a bit more failsafe below, combines these two insights: First, a `sleep` is triggered that allows the scheduler to run other processes, but that is not as precise. Afterwards, this is passed on to busy waiting, that is very precise and will now not be interrupted by the scheduler any more.

The resulting time accuracy is shown here:

<div>
<img src="media/benchmark_sleep_busy_mixed.png" width="700"/>
</div>
This time, the controller did not miss a tick.



Of course, there still are limitations: The function is only a heuristic and gives no guarantees. For a smaller sampling time of $dt = 0.002s$ (five times smaller), also the heuristic reaches its limitations:

<div>
<img src="media/benchmark_mixed_0_002.png" width="700"/>
</div>
The missed ticks were in this run as follows:

| Method | #Missed ticks |
| ------ | ------------- |
| sleep  | 81 |
| busy waiting | 28 |
| mixed | 15| 

As a closing remark, of course some other methods would be possible. One option would be to use a real-time kernel such as the Ubuntu Realtime Kernel. If the code was implemented in C++ instead of Python, there also would be more options such as using the [nanosleep](https://man7.org/linux/man-pages/man2/nanosleep.2.html) function. Both options would however require serious work on the server side of the CloudPendulum and have thus not been implemented yet.

In [ ]:
import warnings
def wait_for_control_loop_end(start_loop, dt, ignore_warnings=False, heuristic=0.7):
    time_to_pass = start_loop + dt - time.time()
    
    if time_to_pass <= 0.0:
        if not(ignore_warnings):
            warnings.warn(f"Attention: The calculation of the control loop took to long - took {time.time() - start_loop} s (that is {time.time() - start_loop - dt} s too long for a {dt} s control loop). " +\
                          "Possible reasons could include that the control loop took too long to calculate, or occassional disruptions on our server.")
        return

    # Combine sleep() and busy waiting for best performance:
    # sleep() gives the scheduler time to perform other tasks, but is quite inaccurate. Busy waiting gives best accuracy, but 
    # keeping the CPU busy at all time results in conflicts with the scheduler of the OS.
    
    time.sleep(time_to_pass * heuristic)
    while time.time() - start_loop < dt:
        pass